### LINEAMIENTOS

La inmobiliaria Properati publica periódicamente información sobre ofertas de propiedades para venta y alquiler. 
Ud. deberá asesorar a la inmobiliaria a desarrollar un modelo de regresión que permita predecir el precio por metro 
cuadrado de una propiedad. El objetivo final, a concretarse en el Desafío 2, es que el modelo que desarrollen sea utilizado 
como tasador automático a ser aplicados a las próximas propiedades que sean comercializadas por la empresa. 
Para ello, la empresa le provee de un dataset correspondiente al primer semestre de 2017.

El dataset es de tamaño entre pequeño y mediano, pero tiene dos complejidades a las que deberá prestarle atención:

● Peso de missing data en algunas variables relevantes.
● Será importante tener en cuenta el problema de la influencia espacial en los precios por metro cuadrado. 
En efecto, es probable que existan diferencias importantes de en las diferentes geografías, barrios y zonas analizadas.

Objetivos:
● Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos 
perdidos en ciertas variables.
● Realizar un análisis descriptivo de las principales variables.
● Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.

Datos: Descargar el dataset de: https://drive.google.com/file/d/0BzVrTKc02N8qNUdDSExBQlFTNlU/view

Requisitos y material a entregar
La limpieza y el análisis de los datos deberán ser entregados en una Jupyter notebook que satisfaga los 
requerimientos del proyecto. La notebook deberá estar debidamente comentada. Además, los grupos deberán crear 
un repositorio para el proyecto en Github.

Para la presentación en clase se deben armar algunos slides no técnicos para exponer en no más de 10 minutos 
(PPT o Google Slides). La presentación debe constar de una introducción (planteo del problema, la pregunta, 
la descripción del dataset, etc.), un desarrollo de los análisis realizados (análisis descriptivo, análisis de correlaciones 
preliminares, visualizaciones preliminares) y una exposición de los principales resultados y conclusiones.

Fecha de entrega
● El material deberá entregarse en la clase 17 del curso, día viernes 13 de septiembre.

Dataset

El dataset contiene información sobre todas las propiedades georeferenciadas de la base de datos de la empresa. 
La información de cada propiedad que incluye es la siguiente:

● Fecha de creación
● Tipo de la propiedad (house, apartment, ph)
● Operación del aviso (sell, rent)
● Nombre del lugar
● Nombre del lugar + nombre de sus ‘padres’
● ID de geonames del lugar (si está disponible)
● Latitud, longitud
● Precio original del aviso
● Moneda original del aviso (ARS, USD)
● Precio del aviso en moneda local (ARS)
● Precio aproximado en USD
● Superficie en m2
● Superficie cubierta en m2
● Precio en USD/m2
● Precio por m2
● N° de piso, si corresponde
● Ambientes
● URL en Properati
● Descripción
● Título
● URL de un thumbnail de la primer foto

¿Cómo empezar? Sugerencias

Agreguen toda otra información construida a partir de los datos originales (o incluso información externa) que consideren 
relevante y útil para resolver los objetivos planteados.

Aprovechen las herramientas de Pandas: groupby, summation, pivot_tables y otras aplicaciones y métodos de los 
DataFrames hacen mucho más simples los cálculos y otras agregaciones de los datos.

En la presentación de los resultados tengan en cuenta que es altamente probable que la audiencia no tenga un nivel técnico, 
así que mantengan el lenguaje en un nivel accesible.

En términos generales, recuerden las siguientes sugerencias:

● Escribir un pseudocódigo antes de empezar a codear. Suele ser muy útil para darle un esquema y una lógica generales 
al análisis.

● Leer la documentación de cualquier tecnología o herramienta de análisis que usen. A veces no hay tutoriales para todo 
y los documentos y las ayudas son fundamentales para entender el funcionamiento de las herramientas utilizadas.

● Documentar todos los pasos, transformaciones, comandos y análisis que realicen.

Recursos útiles

● Documentación de la librería GeoPandas
● Github

Acción: Importo file y realizo sample para tener una primera idea de algunas columnas y sus datos

In [ ]:
import pandas as pd
base = pd.read_csv(r"C:\Users\GuillermoPortero\Desktop\Digital House\Desafio\properatti.csv")
base.sample(5)

Acción: Verifico cómo está conformada la estructura del file.

In [ ]:
base.info()

Observaciones: 
1) Es un Dataframe
2) Son 121220 filas y 26 columnas
3) A priori se observan Labels poco legibles ("Unnamed: 0")
4) Hay poca información para ciertos campos ("lat-lon","expenses","rooms")
5) Habría campos con info repetida ("lat-lon" vs "lat" y "lon"), 6. Hay columnas no relevantes("image...","...url")

Acción: Recorro campo a campo y cuento la cantidad de valores diferentes

In [ ]:
for i in base.columns:
    print("\n'{}' values:\n".format(i),base[i].value_counts())

Observaciones:
#_Unnamed: tiene 121220 valores únicos del tipo integer. Parecería un tipo de índice. A priori no agregaría valor al estimador
#_operation: tiene 121220 valores iguales el cual es "sell". Campo sería descartable.  
#_property_type: tiene 4 valores diferentes (apartment,house,PH,store). Falta corroborar si faltan valores (y cuantos)
#_place_name: tiene muchos valores diferentes que refieren a Localidades, Barrios. Falta corroborar si faltan valores (y cuantos)
#_place_with_parent_names: jerarquía padre/hijo/nieto... Verificar si sirve para complementar info faltante de "place_name" (si faltara) o si no hay concordancia entre dichos campos. Serviría para tener un valor estimado por Provincia, Localidad, Zona
#_country_name: tiene 121220 valores iguales el cual es "Argentina". Campo que no agrega valor.
#_state_name: tiene muchos valores diferentes que refieren Provincia. Falta corroborar si faltan valores (y cuantos). Serviría como base para estimar valores por provincia, y complementarse con place_with_parent_names en aquellos registros que le falte dato.
#_geonames_id: 646 valoes. A priori no agregaría valor al estimador. Utilizaría "lat-lon" para complementar "place_name"
#_'lat-lon': podría complementar a "place_name"
#_lat: a priori sus valores están incluidos en "lat_lon". El campo sería descartable
#_lon: a priori sus valores están incluidos en "lat_lon". El campo sería descartable
#_price: solo 9746 registros con valores. Se requiere de "currency" para interpretarlo. Corroborar si sirve para complementar otros price_usd_per_m2 .
#_currency: ~90k valores útiles. filtar registros que no estén en ARS o USD. Crear nueva columna llevando todo a USD (verificar si existe TC y sino importarlo para usarlo como DICC). 
#_price_aprox_local_currency: ~10.3k valores. Complementa aquellos casos sin "price_usd_per_m2","price", ni "price_aprox_usd" (si es que priorizo estimar en USD frente a ARS)
#_price_aprox_usd: ~10.3k valores. Complementa aquellos casos sin "price_usd_per_m2", ni "price"
#_surface_total_in_m2: ~1.6k valores
#_surface_covered_in_m2: ~1k valores. Principal complementador del precio en dólares para llegar a "price_usd_per_m2"
#_price_usd_per_m2: ~25.6k valores. Principal estimador de precios
#_price_per_m2: ~25k valores. Complementa "price_usd_per_m2" 
#_floor: ~200 valores. A priori no agregaría valor al estimador
#_rooms: calcular cantidad de valores. A priori no agregaría valor al estimador
#_expenses: ~1k. A priori no agregaría valor al estimador
#_properati_url:tiene 121220 valores únicos. Útil para detectar duplicados. Luego descartable
#_description: Campo sería descartable. 
#_title: ~73k. Se podría utilizar para complementar "property_type" o "place_name" ya que varios registros son del tipo "Departamento - Villa Urquiza"
#image_thumbnail: Campo sería descartable.

